In [6]:
from flask import Flask, jsonify
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
old_df = pd.read_csv('ratings_Beauty.csv')


In [8]:
selected_columns = ['user_id', 'product_id', 'rating']
df = old_df[selected_columns]
df = df.drop_duplicates(['user_id', 'product_id', 'rating'])
df['rating'] = df['rating'].astype(float)  # Convert rating column to float
df = df.tail(5000)
# Reset user_id and product_id to sequential numbers starting from 1
df['user_id'] = df['user_id'].astype('category').cat.codes + 1
df['product_id'] = df['product_id'].astype('category').cat.codes + 1
df = df.sort_values('user_id')

df


,user_id,product_id,rating
1047453,1,279,5.0
1044425,2,46,5.0
1046829,3,222,1.0
1046521,4,203,5.0
1047082,5,258,2.0
...,...,...,...
1046944,4938,237,5.0
1044839,4939,114,5.0
1044340,4940,42,4.0
1048343,4941,392,1.0


In [9]:
def recommend_products(user_id):
    pivot_df = df.pivot(index='user_id', columns='product_id', values='rating')
    # Reset the column names
    pivot_df.columns.name = None
    # Reset the index name
    pivot_df.index.name = None
    # Calculate the average rating for each product
    product_avg = pivot_df.mean(axis=0)
    # Fill NaN values with 0
    pivot_df = pivot_df.fillna(0)
    # Calculate the user-product matrix by subtracting the product average from each rating
    matrix_avg = pivot_df.sub(product_avg, axis=1)
    # Calculate the cosine similarity matrix
    similarity = cosine_similarity(matrix_avg)
    # Get the row of the similarity matrix corresponding to the user
    user_similarity = similarity[user_id-1]
    # Find the top n similar users
    similar_users = user_similarity.argsort()[:-2-1:-1]
    # Find the products that the user has not rated
    unrated_products = pivot_df.loc[user_id][pivot_df.loc[user_id] == 0].index
    # Calculate the predicted rating for each unrated product
    product_predictions = {}
    for product in unrated_products:
        product_ratings = matrix_avg[product].values
        # Only consider ratings from similar users
        similar_ratings = product_ratings[similar_users]
        # Ignore ratings of 0 (unrated products)
        similar_ratings = similar_ratings[similar_ratings != 0]
        if len(similar_ratings) > 0:
            # Calculate the predicted rating as the weighted average of similar ratings
            weights = user_similarity[similar_users][similar_ratings.nonzero()]
            prediction = (similar_ratings * weights).sum() / weights.sum()
            product_predictions[product] = prediction
    # Return the top n predicted products
    recommended_products = sorted(
        product_predictions, key=product_predictions.get, reverse=True)
    # return recommended_products
    return recommended_products


In [10]:
for i in range(1, 991):
    recommended_products = recommend_products(i)
    print(f"User ID: {i}, Recommended Products: {recommended_products}")

User ID: 1, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346, 16, 30

User ID: 6, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346, 16, 30

User ID: 11, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346, 

User ID: 16, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346, 

User ID: 21, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346, 16, 3

...

User ID: 966, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346,

User ID: 971, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346,

User ID: 976, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346,

User ID: 981, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346,

User ID: 986, Recommended Products: [66, 68, 82, 94, 161, 217, 225, 313, 351, 354, 362, 364, 369, 379, 384, 392, 395, 37, 57, 60, 61, 81, 84, 95, 144, 146, 149, 177, 178, 182, 226, 250, 290, 295, 156, 23, 311, 170, 399, 301, 164, 25, 31, 55, 83, 88, 91, 92, 104, 109, 113, 129, 136, 142, 148, 163, 167, 180, 185, 193, 195, 207, 221, 252, 263, 266, 267, 268, 271, 304, 314, 324, 325, 331, 353, 371, 376, 262, 121, 157, 79, 134, 28, 132, 12, 209, 337, 17, 38, 192, 261, 212, 389, 39, 47, 151, 162, 189, 211, 269, 299, 307, 356, 381, 390, 327, 85, 222, 128, 13, 53, 69, 108, 141, 191, 235, 11, 126, 231, 291, 208, 345, 219, 87, 152, 210, 355, 203, 220, 228, 298, 338, 215, 8, 20, 22, 32, 34, 35, 44, 58, 62, 71, 77, 78, 80, 89, 90, 93, 96, 101, 119, 123, 133, 166, 184, 186, 216, 218, 238, 265, 274, 276, 281, 283, 285, 289, 305, 309, 320, 334, 341, 343, 347, 360, 375, 382, 317, 393, 110, 329, 43, 190, 24, 36, 122, 127, 310, 197, 51, 272, 397, 279, 257, 378, 131, 278, 319, 328, 383, 86, 213, 67, 346,